In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sympy import sympify, symbols, lambdify

In [ ]:
# input

K1 = 0.1
k2 = 0.12
k3 = 0.05
k4 = 0

AIF_amps = [-2, 1.5, 0.5]
AIF_exps = [4, 1, 0.01]

t_values = np.linspace(0, 120, 1000)  # 500 points between 0 and 10

In [ ]:
t = symbols("t")

if not sum(AIF_amps) == 0:
    raise ValueError("sum of AIF amplitudes must be zero")

In [ ]:
# calculate amplitudes and exponents for the united impulse response function
a1 = (k2 + k3 + k4 - np.sqrt((k2 + k3 + k4) ** 2 - 4 * k2 * k4)) / 2
a2 = (k2 + k3 + k4 + np.sqrt((k2 + k3 + k4) ** 2 - 4 * k2 * k4)) / 2

# calculate the united impulse response function for the 1st and 2nd tissue compartment
UIR1_exps = [a1, a2]
UIR1_amps = [
    K1 * (k4 - a1) / (a2 - a1),
    K1 * (a2 - k4) / (a2 - a1),
]

UIR2_exps = [a1, a2]
UIR2_amps = [
    K1 * k3 / (a2 - a1),
    -K1 * k3 / (a2 - a1),
]

In [ ]:
# setup the united impulse response function (sum of exponentials)

UIR1_str = "+".join(
    [f"{UIR_amp}*exp(-{UIR_exp}*t)" for UIR_amp, UIR_exp in zip(UIR1_amps, UIR1_exps)]
)

UIR1_func = sympify(UIR1_str)
UIR1_func_numeric = lambdify(t, UIR1_func, modules=["numpy"])

UIR2_str = "+".join(
    [f"{UIR_amp}*exp(-{UIR_exp}*t)" for UIR_amp, UIR_exp in zip(UIR2_amps, UIR2_exps)]
)

UIR2_func = sympify(UIR2_str)
UIR2_func_numeric = lambdify(t, UIR2_func, modules=["numpy"])

# calculate the sum of the two impulse response functions
UIR_func = UIR1_func + UIR2_func
UIR_func_numeric = lambdify(t, UIR_func, modules=["numpy"])

In [ ]:
# setup the arterial input function (sum of exponentials)

AIF_str = "+".join(
    [f"{AIF_amp}*exp(-{AIF_exp}*t)" for AIF_amp, AIF_exp in zip(AIF_amps, AIF_exps)]
)

AIF_func = sympify(AIF_str)
AIF_func_numeric = lambdify(t, AIF_func, modules=["numpy"])

In [ ]:
# calculate the response of the 1st compartment
resp1_str_list = []

for (UIR_amp, UIR_exp), (AIF_amp, AIF_exp) in itertools.product(
    zip(UIR1_amps, UIR1_exps), zip(AIF_amps, AIF_exps)
):
    if UIR_exp == AIF_exp:
        tmp = f"{AIF_amp} * {UIR_amp} * t * exp(-{UIR_exp}*t)"
    else:
        tmp = f"({AIF_amp} * {UIR_amp}) * (exp(-{UIR_exp}*t) - exp(-{AIF_exp}*t)) / ({AIF_exp} - {UIR_exp})"

    resp1_str_list.append(tmp)

resp1_str = "+".join(resp1_str_list)
resp1_func = sympify(resp1_str)
resp1_func_numeric = lambdify(t, resp1_func, modules=["numpy"])

In [ ]:
# calculate the response of the 2nd compartment
resp2_str_list = []

for (UIR_amp, UIR_exp), (AIF_amp, AIF_exp) in itertools.product(
    zip(UIR2_amps, UIR2_exps), zip(AIF_amps, AIF_exps)
):
    if UIR_exp == AIF_exp:
        tmp = f"{AIF_amp} * {UIR_amp} * t * exp(-{UIR_exp}*t)"
    else:
        tmp = f"({AIF_amp} * {UIR_amp}) * (exp(-{UIR_exp}*t) - exp(-{AIF_exp}*t)) / ({AIF_exp} - {UIR_exp})"

    resp2_str_list.append(tmp)

resp2_str = "+".join(resp2_str_list)
resp2_func = sympify(resp2_str)
resp2_func_numeric = lambdify(t, resp2_func, modules=["numpy"])

In [ ]:
# calculate the total response of the system
resp_func = resp1_func + resp2_func
resp_func_numeric = lambdify(t, resp_func, modules=["numpy"])

#

In [ ]:
# Plot the function


fig, ax = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True, sharex=True)
if k3 != 0:
    ax[0].plot(
        t_values,
        UIR1_func_numeric(t_values),
        "--",
        label="IR C1",
        color=plt.cm.tab10(1),
    )
    ax[0].plot(
        t_values, UIR2_func_numeric(t_values), ":", label="IR C2", color=plt.cm.tab10(2)
    )
    ax[0].plot(
        t_values,
        UIR_func_numeric(t_values),
        label="IR (C1 + C2)",
        color=plt.cm.tab10(0),
    )
else:
    ax[0].plot(
        t_values, UIR1_func_numeric(t_values), label="IR C1", color=plt.cm.tab10(0)
    )


ax[0].legend()
ax[0].set_ylabel("IR(t)")
ax[0].set_ylim(0, None)
ax[0].set_title(
    f"impulse responses: K1={K1}, k2={k2}, k3={k3}, k4={k4}", fontsize="medium"
)

ax[1].plot(t_values, AIF_func_numeric(t_values), "k")
ax[1].set_ylabel("Ca(t)")
ax[1].set_title(f"arterial input function", fontsize="medium")

if k3 != 0:
    ax[2].plot(
        t_values, resp1_func_numeric(t_values), "--", label="C1", color=plt.cm.tab10(1)
    )
    ax[2].plot(
        t_values, resp2_func_numeric(t_values), ":", label="C2", color=plt.cm.tab10(2)
    )
    ax[2].plot(
        t_values, resp_func_numeric(t_values), label="C1 + C2", color=plt.cm.tab10(0)
    )
else:
    ax[2].plot(
        t_values, resp1_func_numeric(t_values), label="C1", color=plt.cm.tab10(0)
    )
ax[2].legend()
ax[2].set_ylabel("C(t)")
ax[2].set_title(f"system responses", fontsize="medium")

for axx in ax.ravel():
    axx.set_xlabel("t")
    axx.grid(ls=":")
fig.show()